In [1]:
import pandas as pd
import re
import json
import time

In [2]:
df = pd.read_json("hf://datasets/AdaptLLM/finance-tasks/Headline/test.json")

In [3]:
start_time = time.time()

data = df['input'].tolist()

lines = []
for d in data:
    lines.extend(d.split('\n\n'))

res = []
for line in lines:
    if 'Options' in line:
        # 如果句子里有Options,说明句子最后会有选项中的Yes|No, 这并不是要的答案
        info = re.search(r'(.*(Yes|No)) (Yes|No)$', line, re.DOTALL)
    else:
        # 句子里没有Option,默认最后的Yes|No就是答案
        info = re.search(r'(.*?)(Yes|No)$', line, re.DOTALL)
    if info is not None:
        res.append([info.group(1).strip(), info.group(2)])
        
final_result = []
for i, (q, a) in enumerate(res):
    one = {
        'id': i,
        'Question': q,
        'Answer': a
    }
    final_result.append(one)
    
end_time = time.time()

In [4]:
print(f'消耗时间: {end_time - start_time}秒, 提取问答总条数: {len(final_result)}')

消耗时间: 3.0944881439208984秒, 提取问答总条数: 102722


In [5]:
with open('data.json', 'w') as fp:
    json.dump(final_result, fp)